In [1]:
import psycopg2
import pandas as pd 

##### connecting to the main database

In [2]:
try:
    conn = psycopg2.connect(
        database = "postgres",
        user = 'postgres',
        password = 'password',
        host = 'localhost',
        port = '5432'
    )
    conn.set_session(autocommit=True)
    print("Connected to Postgres database succesfully")
    
except psycopg2.Error as e :
    print("Error: Could not make connection to the Postgres database")
    print(e)

Connected to Postgres database succesfully


In [3]:
try :
    cur = conn.cursor()
    print('Initialized cursor succesfully')
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

Initialized cursor succesfully


###### creating the database

In [4]:
try :
    cur.execute('DROP DATABASE IF EXISTS imdb WITH (FORCE)')
    cur.execute('CREATE DATABASE imdb')
    print('Database has been created successfully')
except psycopg2.Error as e:
    print(e)
    

Database has been created successfully


In [5]:
try:
    conn.close()
    print('Connection to the main database closed')
except psycopg2.Error as e:
    print(e)

Connection to the main database closed


##### connecting to the database we just created

In [6]:
try:
    conn = psycopg2.connect(
        database = "imdb",
        user = 'postgres',
        password = 'password',
        host = 'localhost',
        port = '5432'
    )
    conn.set_session(autocommit=True)
    print("Connected to imdb database succesfully")
    
except psycopg2.Error as e :
    print("Error: Could not make connection to the imdb database")
    print(e)

Connected to imdb database succesfully


In [7]:
try :
    cur = conn.cursor()
    print('Initialized cursor succesfully')
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)


Initialized cursor succesfully


##### reading the schema.sql file and executing it to create tables and relations

In [8]:
try:
    cur.execute(open('schema.sql','r').read())
    print('Created the schema succesfully')
except psycopg2.Error as e:
    print(e)

Created the schema succesfully


##### helper functions to insert data from csv file to the required table 
##### and show first five rows of it 

In [9]:
def insert_into_table(table_name):
    with open(table_name+'.csv','r',encoding='utf-8') as f :
        next(f)
        cur.copy_from(f,table_name,sep=',')
        print('Inserted data into {} table successfully'.format(table_name))
    
def show_5_rows(table_name):
    try: 
        cur.execute("SELECT * FROM {} LIMIT 5;".format(table_name))
    except psycopg2.Error as e: 
        print("Error: select *")
        print (e)

    data = cur.fetchall()
    for row in data:
        print(row)

###### inserting data into contents table 

In [10]:
contents_df = pd.read_csv('contents.csv')
contents_df.head(2)

,content_id,title,description,total_seasons,imdb_score,release_dates,play_time,content_rating,total_episodes,content_type,imdb_link,last_updated,imdb_score_votes,rating_details,languages
0,1,Planet Earth II,David Attenborough returns in this breathtakin...,1,9.6,2016,6h,3,1,3,http://www.imdb.com/title/tt5491994/?pf_rd_m=A...,24:09.0,42150,"{'total_votes': 42150, 'females': {'all_votes'...",['English']
1,2,Planet Earth,"Emmy Award-winning, 11 episodes, five years in...",1,9.4,2006,8h 53min,3,1,3,http://www.imdb.com/title/tt0795176/?pf_rd_m=A...,24:12.5,126842,"{'total_votes': 126842, 'females': {'all_votes...",['English']


In [11]:
contents_table_insert = ("""INSERT INTO contents(
content_id,
title,
description,
total_seasons,
imdb_score,
release_dates,
play_time ,
content_rating ,
total_episodes ,
content_type ,
imdb_link,
last_updated,
imdb_score_votes,
rating_details,
languages )
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""") 

In [12]:
for i,row in contents_df.iterrows():
    cur.execute(contents_table_insert,list(row))

In [13]:
show_5_rows('contents')

(1, 'Planet Earth II', 'David Attenborough returns in this breathtaking documentary showcasing life on Planet Earth.', 1, Decimal('9.6'), '2016', '6h', 3, 1, 3, 'http://www.imdb.com/title/tt5491994/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042182&pf_rd_r=0J30GWKBJCBE25N9PV5E&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=toptv&ref_=chttvtp_tt_1', '24:09.0', 42150, "{'total_votes': 42150, 'females': {'all_votes': {'total_votes': 3281, 'rating': 9.7}, 'aged_18_29': {'total_votes': 1807, 'rating': 9.7}, 'aged_under_18': {'total_votes': 24, 'rating': 8.8}, 'aged_30_44': {'total_votes': 868, 'rating': 9.6}, 'aged_over_45': {'total_votes': 222, 'rating': 9.5}}, 'votes_per_ratings': {'3': 59, '2': 89, '8': 2163, '4': 68, '5': 123, '1': 2184, '6': 198, '7': 573, '10': 29740, '9': 6953}, 'all_genders': {'all_votes': {'total_votes': 42150, 'rating': 9.6}, 'aged_18_29': {'total_votes': 16349, 'rating': 9.7}, 'aged_under_18': {'total_votes': 315, 'rating': 9.2}, 'aged_30_44': {'total_votes': 9193, 'rating': 9.6

###### inserting data into episode_list table 

In [14]:
episode_list_df = pd.read_csv('episode_list.csv')
episode_list_df.head(2)

,episode_id,season_num,episode_name,content_id,release_date,episode_rating,episode_num,description,last_updated,episode_imdb_link,episode_score_votes
0,1,1,Pilot,3,20 Jan. 2008,8.9,1,"Diagnosed with terminal lung cancer, chemistry...",2017-12-20 11:24:20.00173,http://www.imdb.com/title/tt0959621/?ref_=ttep...,16425
1,2,1,Cat's in the Bag...,3,27 Jan. 2008,8.7,2,After their first drug deal goes terribly wron...,2017-12-20 11:24:20.008136,http://www.imdb.com/title/tt1054724/?ref_=ttep...,11251


In [15]:
episode_list_table_insert = ("""INSERT INTO episode_list(
episode_id,
season_num,
episode_name,
content_id,
release_date,
episode_rating ,
episode_num,
description ,
last_updated ,
episode_imdb_link ,
episode_score_votes)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
""") 

In [16]:
for i,row in episode_list_df.iterrows():
    cur.execute(episode_list_table_insert,list(row))

In [17]:
show_5_rows('episode_list')

(1, 1, 'Pilot', 3, '20 Jan. 2008', Decimal('8.9'), 1, 'Diagnosed with terminal lung cancer, chemistry teacher Walter White teams up with his former student, Jesse Pinkman, to cook and sell crystal meth.', '2017-12-20 11:24:20.00173', 'http://www.imdb.com/title/tt0959621/?ref_=ttep_ep1', 16425)
(2, 1, "Cat's in the Bag...", 3, '27 Jan. 2008', Decimal('8.7'), 2, "After their first drug deal goes terribly wrong, Walt and Jesse are forced to deal with a corpse and a prisoner. Meanwhile, Skyler grows suspicious of Walt's activities.", '2017-12-20 11:24:20.008136', 'http://www.imdb.com/title/tt1054724/?ref_=ttep_ep2', 11251)
(3, 1, "...And the Bag's in the River", 3, '10 Feb. 2008', Decimal('8.7'), 3, "Walt is struggling to decide if it's best to kill Krazy-8 or let him go.", '2017-12-20 11:24:20.014002', 'http://www.imdb.com/title/tt1054725/?ref_=ttep_ep3', 10903)
(4, 1, 'Cancer Man', 3, '17 Feb. 2008', Decimal('8.3'), 4, 'Walt tells the rest of his family about his cancer. Jesse tries to m

###### inserting data into directors table 

In [18]:
directors_df = pd.read_csv('directors.csv')
directors_df.head(2)

,director_id,name,last_updated
0,1,David Attenborough,2017-12-20 11:24:08.953526
1,2,Sigourney Weaver,2017-12-20 11:24:12.492763


In [19]:
directors_table_insert = ("""INSERT INTO directors(
director_id,
name,
last_updated)
VALUES (%s,%s,%s)
""") 

In [20]:
for i,row in directors_df.iterrows():
    cur.execute(directors_table_insert,list(row))

In [21]:
show_5_rows('directors')

(1, 'David Attenborough', '2017-12-20 11:24:08.953526')
(2, 'Sigourney Weaver', '2017-12-20 11:24:12.492763')
(3, 'Huw Cordey', '2017-12-20 11:24:12.492763')
(4, 'Vince Gilligan', '2017-12-20 11:24:17.96618')
(5, 'High Bridge Productions', '2017-12-20 11:24:17.96618')


###### inserting data into actors table 

In [22]:
insert_into_table('actors')

Inserted data into actors table successfully


In [23]:
show_5_rows('actors')

(1, '"David Attenborough"', '"2017-12-20 11:24:08.953526"')
(2, '"Sigourney Weaver"', '"2017-12-20 11:24:12.492763"')
(3, '"Bryan Cranston"', '"2017-12-20 11:24:17.96618"')
(4, '"Anna Gunn"', '"2017-12-20 11:24:17.96618"')
(5, '"Aaron Paul"', '"2017-12-20 11:24:17.96618"')


###### inserting data into content_actors table 

In [24]:
insert_into_table('content_actors')

Inserted data into content_actors table successfully


In [25]:
show_5_rows('content_actors')

(1, 1, '"2017-12-20 11:24:08.991277"')
(1, 2, '"2017-12-20 11:24:12.532625"')
(2, 2, '"2017-12-20 11:24:12.532625"')
(3, 3, '"2017-12-20 11:24:18.166874"')
(4, 3, '"2017-12-20 11:24:18.166874"')


###### inserting data into content_directors table 

In [26]:
insert_into_table('content_directors')

Inserted data into content_directors table successfully


In [27]:
show_5_rows('content_directors')

(1, 1, '"2017-12-20 11:24:08.991277"')
(1, 2, '"2017-12-20 11:24:12.532625"')
(2, 2, '"2017-12-20 11:24:12.532625"')
(3, 2, '"2017-12-20 11:24:12.532625"')
(4, 3, '"2017-12-20 11:24:18.166874"')


###### inserting data into content_genres table 

In [28]:
insert_into_table('content_genres')

Inserted data into content_genres table successfully


In [29]:
show_5_rows('content_genres')

(7, 1, '24:09.0')
(7, 2, '24:12.5')
(6, 3, '24:18.2')
(8, 3, '24:18.2')
(20, 3, '24:18.2')


###### inserting data into languages table 

In [30]:
insert_into_table('languages')

Inserted data into languages table successfully


In [31]:
show_5_rows('languages')

(1, 'English', '24:09.0')
(2, 'Spanish', '24:18.0')
(3, 'Italian', '24:53.6')
(4, 'Russian', '24:53.6')
(5, 'Dutch', '42:04.7')


##### closing the cursor and connection.


In [32]:
cur.close()
conn.close()